### Chapter 15: Advanced RAG 구축 -1 multiquery + unique-union

In [52]:
import os
from dotenv import load_dotenv
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

if not openai_api_key:
    raise ValueError("openai api 키가 없습니다. 한번더 확인 부탁드립니다.")

os.environ['OPENAI_API_KEY'] = openai_api_key

In [53]:
import bs4
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

#### INDEXING ####

# Load Documents
loader = WebBaseLoader(
    web_paths=("https://news.naver.com/section/101",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("sa_text", "sa_item _SECTION_HEADLINE")
        )
    ),
)
docs = loader.load()

In [54]:
# Split
# text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=20)
# splits = text_splitter.split_documents(docs)

#이점: from_tiktoken_encoder 메서드는 텍스트를 분할할 때 tiktoken 인코더를 사용하여 텍스트를 토큰으로 변환한 다음 분할합니다. 이 방법은 특히 OpenAI 모델과 같은 토큰 기반 언어 모델에서 텍스트를 처리할 때 유용합니다. 
# 토큰 단위로 정확한 분할을 가능하게 하므로, 텍스트가 모델의 입력 토큰 제한에 맞도록 더 정밀하게 분할할 수 있습니다.
# 적용 상황: 이 방법은 모델이 토큰 수를 기준으로 텍스트를 처리해야 할 때(예: GPT 모델) 유리합니다. 특히 텍스트가 다양한 언어를 포함하거나 복잡한 구문을 가질 때 유용합니다.

# Split
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=300, 
    chunk_overlap=50)
splits = text_splitter.split_documents(docs)b

SyntaxError: invalid syntax (1729160929.py, line 13)

In [55]:
# Embed
vectorstore = Chroma.from_documents(documents=splits, 
                                    embedding=OpenAIEmbeddings())

In [56]:
# retriever = vectorstore.as_retriever(search_kwargs={"k": 1})

# MMR 알고리즘이 고려할 문서 수를 더 많이 가져옵니다.
# 그러나 최종적으로 상위 1개 문서만 반환합니다
retriever = vectorstore.as_retriever(
    search_type="mmr", # MMR 알고리즘을 사용하여 검색
    search_kwargs={'k':1,'fetch_k':4} # 상위 1개의 문서를 반환하지만, 고려할 문서는 4개로 설정
)

In [57]:
# Prompt
prompt = hub.pull("sungwoo/ragbasic")

In [58]:
# LLM
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

# Post-processing
def format_docs(docs):
    formatted = "\n\n".join(doc.page_content for doc in docs)
    return formatted

# Chain
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# Question
rag_chain.invoke("국채 관련한 정보를 알려줘")


"국채는 정부가 자금을 조달하기 위해 발행하는 채권으로, 투자자에게 일정 기간 후 원금과 이자를 지급합니다. 최근 정부는 국내투자형 ISA의 의무투자비율을 높여 국내 자산으로의 투자를 유도하고 있습니다. 또한, 외화표시 채권인 '김치본드'의 매입 제한규제를 해제하여 원화 환전 수요를 증가시키고 있습니다."

In [59]:
docs = retriever.get_relevant_documents("국채 관련한 정보를 알려줘")
docs

[Document(metadata={'source': 'https://news.naver.com/section/101'}, page_content="정부가 국내투자형 개인종합자산관리계좌(ISA)의 국내주식 의무투자비율을 높여 국내자산으로의 투자를 유도한다. 또 국내에서 발행되는 외화표시 채권인 '김치본드' 매입 제한규제도 해제해 원화 환전 수요도 늘린다. 해외주\n\n\n한국경제TV")]

In [60]:
# # 문서 검색 시 더 높은 다양성을 가진 문서를 더 많이 검색합니다.
# # 데이터셋에 유사한 문서가 많을 경우 유용합니다.
# docsearch.as_retriever(
#     search_type="mmr",  # MMR(Maximal Marginal Relevance) 알고리즘을 사용하여 검색
#     search_kwargs={'k': 6, 'lambda_mult': 0.25}  # 상위 6개의 문서를 검색하고 다양성을 높이기 위해 lambda 값을 0.25로 설정
# )

# # MMR 알고리즘이 고려할 문서 수를 더 많이 가져옵니다.
# # 그러나 최종적으로 상위 5개 문서만 반환합니다.
# docsearch.as_retriever(
#     search_type="mmr",  # MMR 알고리즘을 사용하여 검색
#     search_kwargs={'k': 5, 'fetch_k': 50}  # 상위 5개의 문서를 반환하지만, 고려할 문서는 50개로 설정
# )

# # 특정 임계값 이상의 유사도 점수를 가진 문서만 검색합니다.
# docsearch.as_retriever(
#     search_type="similarity_score_threshold",  # 유사도 점수 기반 검색
#     search_kwargs={'score_threshold': 0.8}  # 유사도 점수가 0.8 이상인 문서만 검색
# )

# # 데이터셋에서 가장 유사한 문서 하나만 검색합니다.
# docsearch.as_retriever(search_kwargs={'k': 1})

### multiquery + unique-union

# Multi Query
![Example Image](./advancedRagFlow.png)

In [61]:
import bs4
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

#### INDEXING ####

loader = WebBaseLoader(
    web_paths=("https://news.naver.com/section/101",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("sa_text", "sa_item _SECTION_HEADLINE")
        )
    ),
)
docs = loader.load()


# Split
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=300, 
    chunk_overlap=50)

# Make splits
splits = text_splitter.split_documents(docs)


In [62]:
# Index
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
vectorstore = Chroma.from_documents(documents=splits, 
                                    embedding=OpenAIEmbeddings())

retriever = vectorstore.as_retriever()

In [63]:
from langchain.prompts import ChatPromptTemplate

# Multi Query: Different Perspectives
# template = """You are an AI language model assistant. Your task is to generate five 
# different versions of the given user question to retrieve relevant documents from a vector 
# database. By generating multiple perspectives on the user question, your goal is to help
# the user overcome some of the limitations of the distance-based similarity search. 
# Provide these alternative questions separated by newlines. Original question: {question}"""
# prompt_perspectives = ChatPromptTemplate.from_template(template)

template = """
당신은 AI 언어 모델 조수입니다. 당신의 임무는 주어진 사용자 질문에 대해 벡터 데이터베이스에서 관련 문서를 검색할 수 있도록 다섯 가지 다른 버전을 생성하는 것입니다. 
사용자 질문에 대한 여러 관점을 생성함으로써, 거리 기반 유사성 검색의 한계를 극복하는 데 도움을 주는 것이 목표입니다. 
각 질문은 새 줄로 구분하여 제공하세요. 원본 질문: {question}
"""
prompt_perspectives = ChatPromptTemplate.from_template(template)


from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

generate_queries = (
    prompt_perspectives 
    | ChatOpenAI(model_name="gpt-4o-mini",temperature=0) 
    | StrOutputParser() 
    | (lambda x: x.split("\n"))
)

generated_query = generate_queries.invoke("집값의 향방?")
generated_query

['집값의 향후 전망은 어떻게 될까요?',
 '',
 '현재 집값의 추세와 앞으로의 변화 가능성은 무엇인가요?',
 '',
 '부동산 시장에서 집값이 오를지 내릴지에 대한 예측은 어떤가요?',
 '',
 '향후 몇 년간 집값에 영향을 미칠 주요 요인은 무엇인가요?',
 '',
 '집값 상승 또는 하락의 주요 원인은 무엇이며, 앞으로의 방향성은 어떻게 될까요?']

In [64]:
from langchain.load import dumps, loads

def get_unique_union(documents: list[list]):
    """ 고유한 문서들의 합집합을 생성하는 함수입니다. """
    
    # 리스트의 리스트를 평탄화하고, 각 문서를 문자열로 직렬화합니다.
    flattened_docs = [dumps(doc) for sublist in documents for doc in sublist]
    
    # 중복된 문서를 제거하고 고유한 문서만 남깁니다.
    unique_docs = list(set(flattened_docs))
    
    # 고유한 문서를 원래의 문서 객체로 변환하여 반환합니다.
    return [loads(doc) for doc in unique_docs]

In [65]:
# 사용자 질문 정의
question = "집값의 향방?"

In [66]:
# 문서 검색 체인을 구성합니다.
# generate_queries: 주어진 질문에 대해 검색 쿼리를 생성합니다.
# retriever.map(): 생성된 쿼리를 바탕으로 관련 문서를 검색합니다.
# get_unique_union: 검색된 문서에서 중복을 제거하고 고유한 문서들을 반환합니다.
retrieval_chain = generate_queries | retriever.map() | get_unique_union


In [67]:
# 체인을 실행하여 질문에 대한 관련 문서를 검색하고 고유한 문서를 반환합니다.
docs = retrieval_chain.invoke({"question": question})


In [68]:
# 검색된 고유 문서들의 개수를 출력합니다.
len(docs)

# 검색된 고유 문서들을 출력합니다.
docs

[Document(metadata={'source': 'https://news.naver.com/section/101'}, page_content="적극적 자산 효율화로 주주가치 높인 日기업 올림푸스·삿포로홀딩스 등 경영개선 요구에 먼저 '행동' 신규 핵심사업 적극 키우고 잉여 부동산은 과감히 팔아 자산매각 꺼리는 韓 기업들 행동주의 펀드 방어하려면 그들처럼 과\n\n\n매일경제"),
 Document(metadata={'source': 'https://news.naver.com/section/101'}, page_content='디지털타임스\n\n1시간전\n\n\n\n\n\n\n\n\n부동산 시장에 ‘영끌 리턴’.. “집값 불씨 살아났나, 또 터지나?”')]

In [69]:
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough

# RAG
template = """다음 맥락을 바탕으로 질문에 답변하세요:

{context}

질문: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

final_rag_chain = (
    {"context": retrieval_chain, 
     "question": RunnablePassthrough()} 
    | prompt
    | llm
    | StrOutputParser()
)

final_rag_chain.invoke(question)

"최근 부동산 시장에서 '영끌 리턴' 현상이 나타나고 있다는 보도가 있습니다. 이는 집값이 다시 상승할 가능성을 시사하는 것으로 해석될 수 있습니다. 그러나 집값의 향방은 여러 요인에 따라 달라질 수 있으며, 특히 경제 전반의 상황, 금리 변화, 그리고 정부의 정책 등이 중요한 영향을 미칠 것입니다. 따라서 현재의 부동산 시장 동향을 면밀히 살펴보는 것이 필요합니다."